In [102]:
const a = Ref{String}()

Base.RefValue{String}(#undef)

In [106]:
a[] = "hello"

"hello"

In [89]:
import LCMCore
reload("LCMCore")
module comms

import LCMCore: encode, decode, fingerprint

immutable CommsT
    utime::Int64
    format::String
    format_version_major::Int32
    format_version_minor::Int32
    data::Vector{UInt8}
end

fingerprint(::Type{CommsT}) = UInt8[211, 104, 224, 63, 51, 197, 104, 190]

function encode(comms::CommsT)
    io = IOBuffer()
    write(io, fingerprint(CommsT))
    write(io, hton(comms.utime))
    write(io, hton(UInt32(length(comms.format) + 1)))
    write(io, comms.format)
    write(io, UInt8(0))
    write(io, hton(comms.format_version_major))
    write(io, hton(comms.format_version_minor))
    write(io, hton(Int32(length(comms.data))))
    write(io, hton.(comms.data))
    io.data
end

function decode(data::Vector{UInt8}, ::Type{CommsT})
    io = IOBuffer(data)
    msg_fingerprint = read(io, 8)
    if msg_fingerprint != fingerprint(CommsT)
        throw(FingerprintException("LCM message fingerprint did not match. This means that you are trying to decode the wrong message type, or a different version of the message type."))
    end
    utime = Int64(ntoh(read(io, Int64)))
    formatlen = UInt32(ntoh(read(io, UInt32)))
    format = String(read(io, formatlen - 1))
    read(io, 1) # strip off null
    ver_maj = Int32(ntoh(read(io, Int32)))
    ver_min = Int32(ntoh(read(io, Int32)))
    num_bytes = Int32(ntoh(read(io, Int32)))
    data = String(read(io, UInt8, num_bytes))
    CommsT(utime, format, ver_maj, ver_min, data)
end

end

comms

In [90]:
lcm = LCMCore.LCM()

LCMCore.LCM(Ptr{Void} @0x00007fddb87bd4d0,RawFD(131))

In [91]:
function onmsg(channel, msg)
    @show channel msg
end

onmsg (generic function with 1 method)

In [92]:
LCMCore.subscribe(lcm, "HELLO", onmsg, comms.CommsT)

Ptr{Void} @0x00007fddb60ed970

In [101]:
@async LCMCore.handle(lcm)

Task (runnable) @0x000000010d8a4250

channel = "HELLO"
msg = comms.CommsT(12345,"my format",15,3,UInt8[0x7b,0x22,0x68,0x65,0x6c,0x6c,0x6f,0x22,0x3a,0x20,0x22,0x77,0x6f,0x72,0x6c,0x64,0x22,0x7d])


In [100]:
LCMCore.publish(lcm, "HELLO", comms.CommsT(1234567, "format0", 11, 23, "Hello world"))

true

channel = "HELLO"
msg = comms.CommsT(1234567,"format0",11,23,UInt8[0x48,0x65,0x6c,0x6c,0x6f,0x20,0x77,0x6f,0x72,0x6c,0x64])
